https://github.com/sinchir0/lmsys/blob/main/lmsys/exp/exp060.ipynb

# 目的
gemmaを使う、10fold

https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning

In [71]:
# path setting
EXP_NAME = "late-e001-baseline"
MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
COMPETITION_NAME = "atmacup17"

DATA_PATH = "data"
ENV_FILE_PATH = "env_file"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
MODEL_OUTPUT_PATH = f"trained_models/{EXP_NAME}"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True
USE_FOLD = 0
USE_DATA_RATE = 1.0
VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 256
INFERENCE_MAX_LENGTH = 256
SEED = 42
EPOCH = 1
LR = 2e-04
TRAIN_BS = 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 4  # 16
NUM_LABELS = 2

FREEZE_LAYERS = (
    8  # there're 42 layers in total, we don't add adapters to the first 16 layers
)

# rola parameter
LORA_R = 64
LORA_ALPHA = LORA_R * 2
LORA_DROPOUT = 0.05
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする
# TRAINED_MODEL_PATH = "lmsys/trained_models/e006-use-concat"

In [72]:
!nvidia-smi

Sat Aug 31 12:58:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           On  |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             40W /  300W |    7281MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [73]:
!python --version

Python 3.10.14


In [74]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return "kernel", f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return "nohup", f"../../{base_path}"
    elif cwd == f"/content":
        print("Google Colab!")
        return "colab", f"/content/drive/MyDrive/Kaggle/{COMPETITION_NAME}/{base_path}"
    elif cwd.startswith("/home/shinichiro.saito"):
        print("GCP!")
        return "GCP", f"/home/shinichiro.saito/{COMPETITION_NAME}/{base_path}"
    else:
        raise Exception("Unknown environment")


ENV_NAME, DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
_, MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
_, ENV_FILE_PATH = resolve_path(ENV_FILE_PATH)

/home/shinichiro.saito/atmacup17/exp
GCP!
/home/shinichiro.saito/atmacup17/data
/home/shinichiro.saito/atmacup17/exp
GCP!
/home/shinichiro.saito/atmacup17/trained_models/late-e001-baseline
/home/shinichiro.saito/atmacup17/exp
GCP!


In [75]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# install

In [76]:
if ENV_NAME != "GCP":
    %pip install -qq polars==1.0.0
    %pip install -qq transformers==4.42.3
    %pip install -qq datasets==2.20.0
    %pip install -qq evaluate==0.4.2
    %pip install -qq seqeval==1.2.2
    %pip install -qq accelerate==0.32.0
    %pip install -qq python-dotenv==1.0.1
    %pip install -qq wandb==0.17.4
    %pip install -qq bitsandbytes==0.43.1
    %pip install -qq accelerate==0.32.0
    %pip install -qq peft==0.11.1

    # formatter
    %pip install -qq black isort

    %pip install -qq kaggle

# import

In [77]:
import os
import random
import ast
import json

import polars as pl
import numpy as np
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, LayerNorm, MSELoss
import wandb
from datasets import (
    Dataset,
    DatasetDict,
    Value,
    concatenate_datasets,
    load_dataset,
    ClassLabel,
)
from tokenizers import AddedToken
from tqdm.auto import tqdm
from scipy.special import softmax
from sklearn.metrics import log_loss
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    DataCollatorWithPadding,
    TrainingArguments,
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

from sklearn.metrics import log_loss, accuracy_score

In [78]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = os.cpu_count()

In [79]:
import transformers
import datasets
import evaluate
import bitsandbytes
import accelerate
import peft

assert transformers.__version__ == "4.42.3"
assert datasets.__version__ == "2.20.0"
assert evaluate.__version__ == "0.4.2"
assert bitsandbytes.__version__ == "0.43.1"
assert accelerate.__version__ == "0.32.0"
assert peft.__version__ == "0.11.1"

In [80]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [81]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_FILE_PATH}/.env")

True

# Wandb

In [82]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


eval/loss,█▇▄▃▁▁▂▁
eval/roc_auc,▁▅▆█████
eval/runtime,███▆▃▁▁▂
eval/samples_per_second,▁▁▁▃▅██▇
eval/steps_per_second,▁▁▁▄▅███
test/eval_roc_auc,▁
test/loss,▁
test/runtime,▁█
test/samples_per_second,█▁
test/steps_per_second,█▁
train/epoch,▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███


'wandb'

# Data Import & Preprocess

In [13]:
with open(f"{DATA_PATH}/rec_stratified_fold.json") as f:
    label_stratified_fold = json.load(f)

In [40]:
train = (
    pl.read_csv(f"{DATA_PATH}/train_with_index.csv")
    .with_columns(pl.col("Title").fill_null(""), pl.col("Review Text").fill_null(""))
    .with_columns(
        pl.concat_str(["Title", "Review Text"], separator=" ").alias("full_text")
    )
    .rename({"Recommended IND": "labels"})
    .with_columns(  # foldを追加する
        pl.col("index").replace(label_stratified_fold).alias("fold")
    )
)

test = (
    pl.read_csv(f"{DATA_PATH}/test_with_index.csv")
    .with_columns(pl.col("Title").fill_null(""), pl.col("Review Text").fill_null(""))
    .with_columns(
        pl.concat_str(["Title", "Review Text"], separator=" ").alias("full_text")
    )
)

In [41]:
if DEBUG:
    train = train.head(100)

In [63]:
train_dataset = Dataset.from_polars(train)
test_dataset = Dataset.from_polars(test)

In [43]:
train_dataset

Dataset({
    features: ['index', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'labels', 'Positive Feedback Count', 'full_text', 'fold'],
    num_rows: 10000
})

In [44]:
# 計算を早くするために、データを減らす
if not DEBUG:
    train_dataset = train_dataset.select(range(int(len(train_dataset) * USE_DATA_RATE)))

# Model

In [45]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,  # low-rankマトリクスのスケーリングファクター
    # only target self-attention
    # target_modules=["q_proj", "k_proj", "v_proj"],
    target_modules=[  # Linear層を全て含める
        "q_proj",
        "k_proj",
        "v_proj",
        # "o_proj",
        # "gate_proj",
        # "up_proj",
        # "down_proj",
    ],
    layers_to_transform=[i for i in range(42) if i >= FREEZE_LAYERS],
    lora_dropout=LORA_DROPOUT,  # LoRAレイヤーのドロップアウト確率
    bias=LORA_BIAS,
    task_type=TaskType.SEQ_CLS,
)

In [20]:
# tokenizer = GemmaTokenizerFast.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
# tokenizer.add_special_tokens({"additional_special_tokens": ["[SEP]"]})

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
# model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16)
print(model)
print(model.print_trainable_parameters())

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.L

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)

# Tokenize

In [46]:
train_dataset

Dataset({
    features: ['index', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'labels', 'Positive Feedback Count', 'full_text', 'fold'],
    num_rows: 10000
})

In [47]:
def tokenize(example, max_length: int):
    return tokenizer(text=example["full_text"], max_length=max_length, truncation=True)

In [48]:
train_dataset = train_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_length": TRAINING_MAX_LENGTH},
    num_proc=NUM_PROC,
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [49]:
print(train_dataset)

Dataset({
    features: ['index', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'labels', 'Positive Feedback Count', 'full_text', 'fold', 'input_ids', 'attention_mask'],
    num_rows: 10000
})


In [50]:
# def tokenize(examples, max_token_length: int):
#     separator = " [SEP] "

#     joined_text = (
#         examples["last_prompt"]
#         + separator
#         + examples["last_response_a"]
#         + separator
#         + examples["last_response_b"]
#     )

#     return tokenizer(
#         joined_text,
#         max_length=max_token_length,
#         truncation=True,
#         padding="max_length",
#     )


# train_dataset = train_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={"max_token_length": TRAINING_MAX_LENGTH},
#     num_proc=NUM_PROC,
# )

# Train Test Split

In [51]:
filtered_train = train_dataset.filter(
    lambda x: x["fold"] != USE_FOLD, num_proc=NUM_PROC
)
filtered_valid = train_dataset.filter(
    lambda x: x["fold"] == USE_FOLD, num_proc=NUM_PROC
)
# filtered_valid = filtered_valid.select(range(min(VALID_DATA_SIZE, len(filtered_valid))))

train_valid_dataset = DatasetDict(
    {
        "train": filtered_train,
        "valid": filtered_valid,
    }
)

del filtered_train, filtered_valid

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [52]:
print(train_valid_dataset)

DatasetDict({
    train: Dataset({
        features: ['index', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'labels', 'Positive Feedback Count', 'full_text', 'fold', 'input_ids', 'attention_mask'],
        num_rows: 6666
    })
    valid: Dataset({
        features: ['index', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'labels', 'Positive Feedback Count', 'full_text', 'fold', 'input_ids', 'attention_mask'],
        num_rows: 3334
    })
})


In [53]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     preds_prob = softmax(predictions, axis=-1)
#     return {"log_loss": log_loss(labels, preds_prob)}
from sklearn.metrics import roc_auc_score


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_prob = softmax(predictions, axis=-1)
    return {"eval_roc_auc": roc_auc_score(labels, preds_prob[:, 1])}

In [54]:
# スケジューラの設定
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    overwrite_output_dir=True,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=10,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_roc_auc",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",  # "linear", # "constant_with_warmup",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
    optim="adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_valid_dataset["train"],
    # train_dataset=ConcatDataset(train_valid_dataset["train"]),
    eval_dataset=train_valid_dataset["valid"],
    # eval_dataset=ConcatDataset(train_valid_dataset["valid"]),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [55]:
if TRAINING:
    # モデルの学習
    trainer.train(
        resume_from_checkpoint=RESUME_FROM_CHECKPOINT
        if RESUME_FROM_CHECKPOINT
        else None
    )
    # ログの保存に利用したストレージを削除
    # os.system(f"rm -rf {MODEL_OUTPUT_PATH}/checkpoint-*")
    # モデルの保存
    trainer.save_model(MODEL_OUTPUT_PATH)
else:
    # TRAINED_MODEL_PATHを用いて、学習済のモデルを読み込む
    model = AutoModelForSequenceClassification.from_pretrained(
        TRAINED_MODEL_PATH,
        num_labels=NUM_LABELS,
    )
    # model = CustomDebertaSequenceClassification.from_pretrained(MODEL_NAME)

    args = TrainingArguments(
        ".",
        per_device_eval_batch_size=4,
        report_to="none",
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Roc Auc
6,0.471900,0.318646,0.892496
12,0.291000,0.289887,0.940996
18,0.225000,0.226799,0.952603
24,0.206600,0.212600,0.964767
30,0.118400,0.172613,0.968833
36,0.148700,0.181579,0.967615
42,0.246200,0.203116,0.966491
48,0.195600,0.181192,0.969214


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning

```
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
TODO: この　Warningが問題ないのかを調べる

```

# valid_datasetの作成・保存

In [57]:
# TRAININGをINFERRENCEでMAX_TOKENを変えるために、validを作り直す
valid_dataset = train_dataset.filter(
    lambda example: example["index"] in train_valid_dataset["valid"]["index"],
    num_proc=NUM_PROC,
)

valid_dataset = valid_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_length": TRAINING_MAX_LENGTH},
    num_proc=NUM_PROC,
)

# valid_dataset = valid_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={"max_token_length": INFERENCE_MAX_LENGTH},
#     num_proc=NUM_PROC,
# )

# valid_dataset = valid_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={
#         "suffix": "a",
#         "max_token_length": INFERENCE_MAX_LENGTH
#     },
#     num_proc=NUM_PROC,
# ).map(
#     tokenize,
#     batched=False,
#     fn_kwargs={
#         "suffix": "b",
#         "max_token_length": INFERENCE_MAX_LENGTH
#     },
#     num_proc=NUM_PROC,
# )


def add_valid_pred(example, idx, valid_pred):
    example["valid_pred"] = valid_pred[idx]
    return example


valid_dataset = train_valid_dataset["valid"]

valid_pred = softmax(trainer.predict(valid_dataset).predictions, axis=-1)
# valid_pred = softmax(trainer.predict(ConcatDataset(valid_dataset)).predictions, axis=-1)

np.save(f"{MODEL_OUTPUT_PATH}/valid_prediction.npy", valid_pred)

valid_dataset = valid_dataset.map(
    add_valid_pred, with_indices=True, fn_kwargs={"valid_pred": valid_pred}
)

valid_dataset.save_to_disk(f"{MODEL_OUTPUT_PATH}/valid_dataset")

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=8):   0%|          | 0/3334 [00:00<?, ? examples/s]

Map:   0%|          | 0/3334 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3334 [00:00<?, ? examples/s]

# CVの計算

In [61]:
cv_score = roc_auc_score(valid_dataset["labels"], valid_pred[:, 1])
print(f"CV Score: {cv_score}")

CV Score: 0.9694181297244575


In [62]:
# output_textを保存
with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
    f.write(str(cv_score))

# テストの計算

In [66]:
test_dataset = test_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_length": INFERENCE_MAX_LENGTH},
    num_proc=NUM_PROC,
)


def add_valid_pred(example, idx, valid_pred):
    example["valid_pred"] = valid_pred[idx]
    return example


test_pred = softmax(trainer.predict(test_dataset).predictions, axis=-1)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=8):   0%|          | 0/11155 [00:00<?, ? examples/s]

In [68]:
MODEL_OUTPUT_PATH

'/home/shinichiro.saito/atmacup17/trained_models/e060-10fold'

In [84]:
import os

os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)
sample_submission = pl.read_csv(f"{DATA_PATH}/sample_submission.csv")

(
    sample_submission.with_columns(
        pl.Series(test_pred[:, 1]).alias("target")
    ).write_csv(f"{MODEL_OUTPUT_PATH}/submission_{EXP_NAME}_cv{cv_score:.4f}.csv")
)

# AWSへのアップロード

In [ ]:
# S3へのアップロード
# TODO: colabでは動かないため直す
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [ ]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [ ]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp /{DATA_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [ ]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e060-10fold-gemma-2-9b-it-bnb-4bit, output_dir:/home/shinichiro.saito/lmsys/trained_models/e060-10fold
Starting upload for file tokenizer.json


  0%|          | 0.00/16.7M [00:00<?, ?B/s]

100%|██████████| 16.7M/16.7M [00:00<00:00, 25.6MB/s]


Upload successful: tokenizer.json (17MB)
Starting upload for file training_args.bin


100%|██████████| 5.12k/5.12k [00:00<00:00, 14.6kB/s]


...

Upload successful: tokenizer_config.json (40KB)
Starting upload for file valid_prediction.npy


100%|██████████| 35.3k/35.3k [00:00<00:00, 89.9kB/s]


Upload successful: valid_prediction.npy (35KB)
Starting upload for file cv_score.txt


100%|██████████| 18.0/18.0 [00:00<00:00, 48.9B/s]


Upload successful: cv_score.txt (18B)


# ローカルからのデータの削除

In [ ]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [ ]:
if WANDB:
    wandb.finish()

In [ ]:
print("finish Notebook!")